In [10]:
import re

def tokenize_arithmetic_expression(expression):
    # Define regular expressions for tokens
    token_patterns = [
        (r'\d+', 'NUMBER'),           # Match one or more digits as numbers
        (r'[+\-*/^()]', 'OPERATOR'),  # Match arithmetic operators and parentheses
    ]

    tokens = []  # List to store tokens

    # Tokenize the expression
    pos = 0
    while pos < len(expression):
        found_match = False
        for pattern, token_type in token_patterns:
            match = re.match(pattern, expression[pos:])
            if match:
                found_match = True
                tokens.append((match.group(), token_type))
                pos += len(match.group())
                break
        if not found_match:
            raise ValueError("Invalid character found at position", pos)

    return tokens

def accept_arithmetic_expression():
    while True:
        try:
            expression = input("Enter an arithmetic expression: ")
            tokens = tokenize_arithmetic_expression(expression)
            print("Tokens:", tokens)
            break
        except ValueError as e:
            print("Error:", e)

# Example usage
accept_arithmetic_expression()


Enter an arithmetic expression: 4*5
Tokens: [('4', 'NUMBER'), ('*', 'OPERATOR'), ('5', 'NUMBER')]


In [29]:
import re

class Node:
    def __init__(self, value):
        self.value = value
        self.children = []

    def add_child(self, node):
        self.children.append(node)

    def evaluate(self):
        if self.value.isdigit():
            return int(self.value)
        elif self.value in ['+', '-', '*', '/']:
            left_value = self.children[0].evaluate()
            right_value = self.children[1].evaluate()
            if self.value == '+':
                return left_value + right_value
            elif self.value == '-':
                return left_value - right_value
            elif self.value == '*':
                return left_value * right_value
            elif self.value == '/':
                return left_value / right_value
        elif self.value == '^':
            base = self.children[0].evaluate()
            exponent = self.children[1].evaluate()
            return base ** exponent
        elif self.value in ['+', '-']:
            return self.children[0].evaluate()
        else:
            raise ValueError("Invalid node value")

class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.current_token = None
        self.index = -1
        self.advance()

    def advance(self):
        self.index += 1
        if self.index < len(self.tokens):
            self.current_token = self.tokens[self.index]
        else:
            self.current_token = None

    def parse(self):
        return self.parse_expr()

    def parse_expr(self):
        node = self.parse_term()

        # Check if there's only one term
        if not self.current_token or self.current_token[1] not in ['+', '-']:
            return node

        while self.current_token and self.current_token[1] in ['+', '-']:
            op = self.current_token
            self.advance()
            right = self.parse_term()
            expr_node = Node(op[0])
            expr_node.add_child(node)
            expr_node.add_child(right)
            node = expr_node

        return node


    def parse_term(self):
        node = self.parse_factor()

        while self.current_token and self.current_token[1] in ['*', '/']:
            op = self.current_token
            self.advance()
            right = self.parse_factor()
            term_node = Node(op[0])
            term_node.add_child(node)
            term_node.add_child(right)
            node = term_node

        return node

    def parse_factor(self):
        node = self.parse_power()

        while self.current_token and self.current_token[0] == '^':
            op = self.current_token
            self.advance()
            right = self.parse_power()
            factor_node = Node(op[0])
            factor_node.add_child(node)
            factor_node.add_child(right)
            node = factor_node

        return node

    def parse_power(self):
        if self.current_token[0] in ['+', '-']:
            unary_op = self.current_token
            self.advance()
            primary_node = self.parse_primary()
            unary_node = Node(unary_op[0])
            unary_node.add_child(primary_node)
            return unary_node
        else:
            return self.parse_primary()

    def parse_primary(self):
        if self.current_token[0] == '(':
            self.advance()
            expr_node = self.parse_expr()
            if self.current_token[0] == ')':
                self.advance()
                return expr_node
            else:
                raise SyntaxError("Expected closing parenthesis")
        elif self.current_token[1] == 'NUMBER':
            node = Node(self.current_token[0])
            self.advance()
            return node
        else:
            raise SyntaxError("Unexpected token: " + self.current_token[0])


def tokenize_arithmetic_expression(expression):
    # Define regular expressions for tokens
    token_patterns = [
        (r'\d+', 'NUMBER'),           # Match one or more digits as numbers
        (r'[+\-*/^()]', 'OPERATOR'),  # Match arithmetic operators and parentheses
    ]

    tokens = []  # List to store tokens

    # Tokenize the expression
    pos = 0
    while pos < len(expression):
        found_match = False
        for pattern, token_type in token_patterns:
            match = re.match(pattern, expression[pos:])
            if match:
                found_match = True
                tokens.append((match.group(), token_type))
                pos += len(match.group())
                break
        if not found_match:
            raise ValueError("Invalid character found at position", pos)

    return tokens

def accept_arithmetic_expression():
    while True:
        try:
            expression = input("Enter an arithmetic expression: ")
            tokens = tokenize_arithmetic_expression(expression)
            print("Tokens:", tokens)

            # Create a parser object and parse the tokens
            parser = Parser(tokens)
            parse_tree = parser.parse()
            print("Parse Tree:", parse_tree)

            # Evaluate the parse tree to get the result
            result = parse_tree.evaluate()
            print("Result:", result)

            break
        except ValueError as e:
            print("Error:", e)

# Example usage
accept_arithmetic_expression()


Enter an arithmetic expression: 2-3
Tokens: [('2', 'NUMBER'), ('-', 'OPERATOR'), ('3', 'NUMBER')]
Parse Tree: <__main__.Node object at 0x7e265c70b400>
Result: 2
